In [1]:
using JuMP, Gurobi, CSV, DataFrames

In [10]:
"""
"VERY_EARLY_MORNING",
"EARLY_AM",
"AM_PEAK",
"MIDDAY_SCHOOL",
"MIDDAY_BASE",
"PM_PEAK", 
"EVENING", 
"LATE_EVENING", 
"NIGHT"
""";

### Data loading

In [16]:
Blue1 = Matrix(CSV.read("processed_data/Blue1_dataset.csv", DataFrame));
Blue2 = Matrix(CSV.read("processed_data/Blue2_dataset.csv", DataFrame));

Orange1 = Matrix(CSV.read("processed_data/Orange1_dataset.csv", DataFrame));
Orange2 = Matrix(CSV.read("processed_data/Orange2_dataset.csv", DataFrame));

Green1 = Matrix(CSV.read("processed_data/Green1_dataset.csv", DataFrame));
Green2 = Matrix(CSV.read("processed_data/Green2_dataset.csv", DataFrame));

Red1 = Matrix(CSV.read("processed_data/Red1_dataset.csv", DataFrame));
Red2 = Matrix(CSV.read("processed_data/Red2_dataset.csv", DataFrame));

In [18]:
AvgFlowBlue = cat(Blue1, Blue2, dims = 3);
AvgFlowOrange = cat(Orange1, Orange2, dims = 3);
AvgFlowGreen = cat(Green1, Green2, dims = 3);
AvgFlowRed = cat(Red1, Red2, dims = 3);

In [20]:
cost_blue = 10000;
cost_orange = 9000;
cost_green = 8000;
cost_red = 10000;

capacity_blue = 1000;
capacity_orange = 1100;
capacity_green = 900;
capacity_red = 1100;

q = 2.4;

## Blue model

In [22]:
stations_b, times, directions = size(AvgFlowBlue);

In [62]:
modelBlue = Model(Gurobi.Optimizer)

@variable(modelBlue, x[1:directions, 1:times] >= 0, Int)
@variable(modelBlue, u[1:directions, 1:times, 1:stations_b] >= 0)
@variable(modelBlue, s[1:directions, 1:times] >= 0, Int)
@variable(modelBlue, r[1:directions, 1:times], Int)

@constraint(modelBlue, [d=1:directions, j=1:times, i=1:stations_b], 
    u[d,j,i] + capacity_blue * (x[d,j] + s[d,j]) >= AvgFlowBlue[i,j,d])

@constraint(modelBlue,  r[2,1] ==   x[1,1] )
@constraint(modelBlue,  r[1,1] ==  x[2,1]  ) 
@constraint(modelBlue, [j=2:times], r[2,j] ==   x[1,j] +  s[1,j]  - s[2,j] + r[2,j-1])
@constraint(modelBlue, [j=2:times], r[1,j] ==  x[2,j] +  s[2,j] - s[1,j] + r[1,j-1])

@constraint(modelBlue, [d=1:directions], s[d,1] == 0)
@constraint(modelBlue, [d=1:directions, j=2:times], s[d,j] <= r[d,j-1])

@constraint(modelBlue, [d=1:directions], r[1, times]  ==  x[1,1])
@constraint(modelBlue, [d=1:directions], r[2, times]  ==  x[2,1])

@constraint(modelBlue, [j=1:times, d=1:directions], x[d,j] + s[d,j] >= 1)

@objective(modelBlue, Min, sum(sum(cost_blue * x[d,j] + 0.9 * cost_blue * s[d,j] + 
            sum(q * u[d,j,i] for i=1:stations_b) for d=1:directions) for j=1:times))

optimize!(modelBlue)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 274 rows, 270 columns and 810 nonzeros
Model fingerprint: 0x0f54d931
Variable types: 216 continuous, 54 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 225 rows and 125 columns
Presolve time: 0.00s
Presolved: 49 rows, 145 columns, 242 nonzeros
Variable types: 96 continuous, 49 integer (0 binary)

Root relaxation: objective 6.480272e+05, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 648027.200    0   29          - 648027.200      -    

## Orange model

In [67]:
stations_o, times, directions = size(AvgFlowOrange);

In [69]:
modelOrange = Model(Gurobi.Optimizer)

@variable(modelOrange, x[1:directions, 1:times] >= 0, Int)
@variable(modelOrange, u[1:directions, 1:times, 1:stations_o] >= 0)
@variable(modelOrange, s[1:directions, 1:times] >= 0, Int)
@variable(modelOrange, r[1:directions, 1:times], Int)

@constraint(modelOrange, [d=1:directions, j=1:times, i=1:stations_o], 
    u[d,j,i] + capacity_orange * (x[d,j] + s[d,j]) >= AvgFlowOrange[i,j,d])

@constraint(modelOrange,  r[2,1] ==   x[1,1] )
@constraint(modelOrange,  r[1,1] ==  x[2,1]  ) 
@constraint(modelOrange, [j=2:times], r[2,j] ==   x[1,j] +  s[1,j]  - s[2,j] + r[2,j-1])
@constraint(modelOrange, [j=2:times], r[1,j] ==  x[2,j] +  s[2,j] - s[1,j] + r[1,j-1])

@constraint(modelOrange, [d=1:directions], s[d,1] == 0)
@constraint(modelOrange, [d=1:directions, j=2:times], s[d,j] <= r[d,j-1])

@constraint(modelOrange, [d=1:directions], r[1, times]  ==  x[1,1])
@constraint(modelOrange, [d=1:directions], r[2, times]  ==  x[2,1])

@constraint(modelOrange, [j=1:times, d=1:directions], x[d,j] + s[d,j] >= 1)

@objective(modelOrange, Min, sum(sum(cost_orange * x[d,j] + 0.9 * cost_orange * s[d,j] + 
            sum(q * u[d,j,i] for i=1:stations_o) for d=1:directions) for j=1:times))

optimize!(modelOrange)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 418 rows, 414 columns and 1242 nonzeros
Model fingerprint: 0xc122e25f
Variable types: 360 continuous, 54 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 370 rows and 155 columns
Presolve time: 0.00s
Presolved: 48 rows, 259 columns, 355 nonzeros
Variable types: 211 continuous, 48 integer (0 binary)

Root relaxation: objective 9.404336e+05, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 940433.564    0   29          - 940433.564      -  

## Red model

In [73]:
stations_r, times, directions = size(AvgFlowRed);
lines = 2;

In [74]:
Red1 = ["Alewife", "Davis", "Porter", "Harvard", "Central", "Kendall/MIT", "Charles/MGH",
     "Park Street", "Downtown Crossing", "South Station", "Broadway", "Andrew", "JFK/Umass",
      "Savin Hill", "Fields Corner", "Shawmut", "Ashmont"];

Red2 = ["Alewife", "Davis", "Porter", "Harvard", "Central", "Kendall/MIT", "Charles/MGH",
     "Park Street", "Downtown Crossing", "South Station", "Broadway", "Andrew", "JFK/Umass",
      "North Quincy", "Wollaston", "Quincy Center", "Quincy Adams", "Braintree"];

In [75]:
z_red = zeros((2,22));

for i = 1:22
    if red_stations[i] in Red1
        z_red[1,i] = 1
    end
    if red_stations[i] in Red2
        z_red[2,i] = 1
    end
end

In [79]:
modelRed = Model(Gurobi.Optimizer)

@variable(modelRed, x[1:directions, 1:times, 1:lines] >= 0)
@variable(modelRed, u[1:directions, 1:times, 1:stations_r] >= 0)
@variable(modelRed, s[1:directions, 1:times, 1:stations_r] >= 0, Int)
@variable(modelRed, r[1:directions, 1:times, 1:stations_r], Int)

@constraint(modelRed, [d=1:directions, j=1:times, i=1:stations_r], 
    u[d,j,i] + (sum(capacity_red * (x[d,j,l] + s[d,j,l]) * z_red[l,i] for l=1:lines)) >= AvgFlowRed[i,j,d])

@constraint(modelRed, [l=1:lines], r[2,1,l] ==   x[1,1,l] )
@constraint(modelRed, [l=1:lines], r[1,1,l] ==  x[2,1,l]  ) 
@constraint(modelRed, [j=2:times, l=1:lines], r[2,j,l] ==   x[1,j,l] +  s[1,j,l]  - s[2,j,l] + r[2,j-1,l])
@constraint(modelRed, [j=2:times, l=1:lines], r[1,j,l] ==  x[2,j,l] +  s[2,j,l] - s[1,j,l] + r[1,j-1,l])

@constraint(modelRed, [d=1:directions, l=1:lines], s[d,1,l] == 0)
@constraint(modelRed, [d=1:directions, j=2:times, l=1:lines], s[d,j,l] <= r[d,j-1,l])

@constraint(modelRed, [d=1:directions, l=1:lines], r[1, times, l]  ==  x[1,1,l])
@constraint(modelRed, [d=1:directions, l=1:lines], r[2, times, l]  ==  x[2,1,l])

@constraint(modelRed, [j=1:times, d=1:directions, l=1:lines], x[d,j,l] + s[d,j,l] >= 1)

@objective(modelRed, Min, sum( sum(cost_red * x[d,j,l] + 0.9 * cost_red * s[d,j,l] for l=1:lines ) + 
                sum(q * u[d,j,i] for i=1:stations_r) for d=1:directions,  j=1:times))

optimize!(modelRed)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 512 rows, 1224 columns and 720 nonzeros
Model fingerprint: 0x1e028d29
Variable types: 432 continuous, 792 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 464 rows and 1176 columns
Presolve time: 0.00s
Presolved: 48 rows, 48 columns, 144 nonzeros
Variable types: 0 continuous, 48 integer (0 binary)

Root relaxation: objective 3.294282e+06, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3294282.4000 3294282.40  0.00%   

### Green line

In [84]:
stations_g, times, directions = size(AvgFlowGreen);
lines = 4;

In [85]:
Green1 = ["Lechmere", "Science Park", "North Station", "Haymarket", "Government Center", "Park Street", "Boylston",
 "Arlington", "Copley", "Hynes Convention Center", "Kenmore", "Blandford Street", "Boston Univ. East", 
 "Boston Univ. Central", "Boston Univ. West", "Saint Paul Street", "Pleasant Street", "Babcock Street",
 "Harvard Ave.","Griggs Street", "Allston Street", "Warren Street", "Washington Street", "Sutherland Road",
"Chiswick Road", "Chestnut Hill Ave.", "South Street"];

Green2 = ["Government Center", "Park Street", "Boylston", "Arlington", "Copley", "Hynes Convention Center", "Kenmore", 
"Saint Mary Street", "Hawes Street", "Kent Street", "Saint Paul Street", "Coolidge Corner", "Summit Ave.", 
"Brandon Hall", "Fairbanks Street", "Washington Square", "Tappan Street", "Dean Road", "Englewood Ave.", "Cleveland Circle"];

Green3 = ["Lechmere", "Science Park", "North Station", "Haymarket", "Government Center", "Park Street", "Boylston", 
"Arlington", "Copley", "Hynes Convention Center", "Kenmore", "Fenway", "Longwood", "Brookline Village", "Brookline Hills",
"Beaconsfield", "Reservoir", "Chestnut Hill Ave.", "Newton Centre", "Newton Highlands", "Eliot", "Waban", "Woodland", 
"Riverside"];

Green4 = ["Lechmere", "Science Park", "North Station", "Haymarket", "Government Center", "Park Street", "Boylston",
    "Arlington", "Copley", "Prudential", "Symphony", "Northeastern University", "Museum of Fine Arts", "Longwood",
    "Brigham Circle", "Fenwood Road", "Mission Park", "Riverway", "Back of the Hill", "Heath Street"];


In [86]:
z_green = zeros((4,66));

for i = 1:66
    if green_stations[i] in Green1
        z_green[1,i] = 1
    end
    if green_stations[i] in Green2
        z_green[2,i] = 1
    end
    if green_stations[i] in Green3 
        z_green[3,i] = 1
    end
    if green_stations[i] in Green4 
        z_green[4,i] = 1
    end
end

In [88]:
modelGreen = Model(Gurobi.Optimizer)

@variable(modelGreen, x[1:directions, 1:times, 1:lines] >= 0)
@variable(modelGreen, u[1:directions, 1:times, 1:stations_g] >= 0)
@variable(modelGreen, s[1:directions, 1:times, 1:stations_g] >= 0, Int)
@variable(modelGreen, r[1:directions, 1:times, 1:stations_g], Int)

@constraint(modelGreen, [d=1:directions, j=1:times, i=1:stations_g], 
    u[d,j,i] + (sum(capacity_green * (x[d,j,l] + s[d,j,l]) * z_green[l,i] for l=1:lines)) >= AvgFlowGreen[i,j,d])

@constraint(modelGreen, [l=1:lines], r[2,1,l] ==   x[1,1,l] )
@constraint(modelGreen, [l=1:lines], r[1,1,l] ==  x[2,1,l]  ) 
@constraint(modelGreen, [j=2:times, l=1:lines], r[2,j,l] ==   x[1,j,l] +  s[1,j,l]  - s[2,j,l] + r[2,j-1,l])
@constraint(modelGreen, [j=2:times, l=1:lines], r[1,j,l] ==  x[2,j,l] +  s[2,j,l] - s[1,j,l] + r[1,j-1,l])

@constraint(modelGreen, [d=1:directions, l=1:lines], s[d,1,l] == 0)
@constraint(modelGreen, [d=1:directions, j=2:times, l=1:lines], s[d,j,l] <= r[d,j-1,l])

@constraint(modelGreen, [d=1:directions, l=1:lines], r[1, times, l]  ==  x[1,1,l])
@constraint(modelGreen, [d=1:directions, l=1:lines], r[2, times, l]  ==  x[2,1,l])

@constraint(modelGreen, [j=1:times, d=1:directions, l=1:lines], x[d,j,l] + s[d,j,l] >= 1)

@objective(modelGreen, Min, sum( sum(cost_green * x[d,j,l] + 0.9 * cost_green * s[d,j,l] for l=1:lines ) + 
                sum(q * u[d,j,i] for i=1:stations_g) for d=1:directions,  j=1:times))

optimize!(modelGreen)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1420 rows, 3636 columns and 1836 nonzeros
Model fingerprint: 0x8ffabb2d
Variable types: 1260 continuous, 2376 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 1372 rows and 3588 columns
Presolve time: 0.00s
Presolved: 48 rows, 48 columns, 144 nonzeros
Variable types: 0 continuous, 48 integer (0 binary)

Root relaxation: objective 2.159008e+06, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2159008.0000 2159008.00  0.0

## Robust Version